In [ ]:
import pandas as pd
import numpy as np
import re
import datetime as time
from mlxtend.frequent_patterns import apriori, association_rules
import sys
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)
df = pd.read_parquet('scontrini_2.parquet',engine='pyarrow')
df = df[(~df.reparto.isin(["95","80","85","03"]))&(df.tipo=="")]
df

In [ ]:
def hot_encode(x):
    if(x==0):
        return False
    else:
        return True

ho differenziato i prodotti scontati da quelli non scontati aggiungendo il tipo di sconto alla fine della descrizione

In [ ]:
df['descrizione']=(df['descrizione']+" "+df['tipo_promo']).str.strip()

In [ ]:
sconti=df[df.promoNoPromo!=0]

In [ ]:
transazioni=sconti[['cassa','transazione','data_reg','descrizione','qta_riga']]

basket=transazioni.groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack()\
    .reset_index()\
    .fillna(0)\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)
basket

si ottiene che se le banane sono in sconto si comprano anche le zucchine

In [ ]:
frq_items = apriori(basket, min_support = 0.001, use_colnames = True)
rules = association_rules(frq_items, metric ="confidence", min_threshold = 0.01)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(frq_items.sort_values(by=['support'], ascending=False))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules.sort_values(by=['confidence'], ascending=False))